# Calculating sums and averages
Workflow that describes how the statistics over a set of intervals / zones can be calculated in Python

Necessary imports

In [1]:
import os
import utils.io as uio
import core.well as cw
from core.well import Project

## Create a wells project

In [3]:
wp = Project(name='MyProject', tops_file='test_data/RokDocTops.xlsx', tops_type='rokdoc')

*Project* also take keywords:
- *working_dir* - the folder where the project is situated
- *project_table* - full or relative path name of a specific formatted .xlsx table that contain important project information
- *log_to_stdout* - if True, logging information is sent to standard output, else to a text file

By default, none of these need to set, and it uses the install directory, toghether with example data and project table there

## Load project table
The project table is a .xlsx file that contain, among other things, which wells and well logs to load, and where to find them 

In [4]:
well_table = uio.project_wells(wp.project_table, wp.working_dir)

## Load tops
Load file with well tops. They can be in format exported from Petrel (use uio.read_petrel_tops()), npd.no (use uio.read_npd_tops()), or RokDoc (use uio.read_rokdoc_tops())


In [5]:
tops = uio.read_tops(wp.tops_file, frmt=wp.tops_type)
#print('The tops file contains following wells and tops:')
for well in list(tops.keys()):
    print(' Well: {}'.format(well))
#    for ttop in list(tops[well].keys()):
#        print('  - {}'.format(ttop))

 Well: WELL_A
 Well: WELL_C
 Well: WELL_B
 Well: WELL_F


   # Calculate RokDoc compatible Sums & Averages
  This functionality is useful when you want to analyze the statistics of several formations / intervals across multiple wells (.las files).
  The results is saved as an .xlsx spreadsheet, with statistics of the rock properties. To be read by RokDoc, it has to be converted to .xls

Tell python where to save the results, where to find the wells, and the tops file

If the .xlsx file exists, it will be appended to
If you want to save the results, or load the data from, elsewhere on your file system, please provide the full path name, using "forward slashes"
In the folder of where the .xlsx file is situated, all QC plots will be saved. Their name will end with the below 'tag'

In [6]:
rd_file = 'results_folder/RokDoc_SumsAverages_from_python.xlsx'

tag = 'my_test'

Modify the project_table.xlsx so that it points to the wells; Well_A, Well_B and Well_C

<img src="images/Annotation 2020-02-11 093608.png" />


The log name(s) under each log types *P velocity*, *S velocity*, *Density*, *Porosity*, and *Volume* must be specified.

Multiple log names can be specified under each log type **BUT only the first log name under each log type will be used in the proceeding analysis**

Read in the project table again to make sure any changes are taken into account

In [7]:
well_table = uio.project_wells(wp.project_table, wp.working_dir)

Define the formations / intervals you like to calculate the statistics over.
 - 'name' is an arbitrary name used when writing the sums & averages to file
 - 'tops' is a list of formation / interval names of the top and base that defines the interval of interest. For the calculation to work, these names have to exist in the above defined tops_file, for the given wells. 

In [8]:
intervals_sands = [
            {'name': 'Sand H',
             'tops': ['Top H', 'Base H']},        
            {'name': 'Sand F',
             'tops': ['Top F', 'Base F']},
            {'name': 'Sand E',
             'tops': ['Top E', 'Top F']},
            {'name': 'Sand D',
             'tops': ['Top D', 'Top E']},
            {'name': 'Sand C',
             'tops': ['Top C', 'Top D']}
]
intervals_shales = [
    {'name': 'Shale C',
         'tops': ['Top C', 'Top D']},
    {'name': 'Shale G',
         'tops': ['Top G', 'Top H']}
]

Test to see if the above intervals are present in all wells

In [10]:
for intervals in [intervals_sands, intervals_shales]:
    for key in list(well_table.keys()):
        well_name = well_table[key]['Given well name']
        for top_name in [x['tops'][0].upper() for x in intervals]:
            if top_name not in list(tops[well_name].keys()):
                print('Top: {}, not present in well {}'.format(top_name, well_name))
        for base_name in [x['tops'][1].upper() for x in intervals]:
            if base_name not in list(tops[well_name].keys()):
                print('Base: {}, not present in well {}'.format(base_name, well_name))

Define the cut offs that are used to classify the data (e.g. sands or shales).
The statistics will only be calculated within each interval, where the cut off is valid
The log names (e.g. *vcl*) corresponding to a log type (e.g. *Volume*) must exist in the .las files

In [11]:
cutoffs_sands = {'Volume': ['<', 0.5], 'Porosity': ['>', 0.1]}
cutoffs_shales = {'Volume': ['>', 0.5], 'Porosity': ['<', 0.1]}

Run the calculation of the statistics

In [ ]:
cw.calc_stats(well_table, tops, intervals_sands, cutoffs_sands, 
              rokdoc_output=rd_file,
              working_dir=os.path.join(wp.working_dir, 'results_folder'),
              suffix=tag)

In [ ]:
cw.calc_stats(well_table, tops, intervals_shales, cutoffs_shales, 
              rokdoc_output=rd_file,
              working_dir=os.path.join(wp.working_dir, 'results_folder'),
              suffix=tag)

Interval: Shale C
 Well: WELL_A
   Interval Shale C, in well WELL_A, has the depth range: 1585.1 - 1820.7
 Well: WELL_B
   Interval Shale C, in well WELL_B, has the depth range: 1787.8 - 1948.0
 Well: WELL_C
   Interval Shale C, in well WELL_C, has the depth range: 765.0 - 1763.9
Appending to existing RokDoc Sums and averages file
Interval: Shale G
 Well: WELL_A
   Interval Shale G, in well WELL_A, has the depth range: 2158.1 - 2211.0
 Well: WELL_B
   Interval Shale G, in well WELL_B, has the depth range: 2656.1 - 2691.9
 Well: WELL_C
   Interval Shale G, in well WELL_C, has the depth range: 2024.3 - 2067.4
Appending to existing RokDoc Sums and averages file


## Test
Create a Well object, and read in one las file

In [ ]:
from core.well import Well
w = Well()
w.read_well_table(well_table, 0)
w.depth_plot('Volume', tops=tops)

In [ ]:
w.log_blocks['Log block 1'].logs.keys()